<a href="https://www.kaggle.com/code/andrapsrin/datasets-join?scriptVersionId=170946062" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Step 1: Data Reading

In order to read the data, we need to handle entries that have commas inside the content of one of the columns. This causes the csv format to throw an error while parsing the data because it can’t distinguish between commas in text versus commas for delimiting columns. An easy fix would be to check if the comma is found between " " (this is the case for the text columns), but not all values fit this requirement so it becomes difficult to differentiate the commas. 

To have a clear understanding of the innacurate data, the number of skipped lines will be printed.

In [1]:
import pandas as pd
import numpy as np

# count the number of lines skipped in google dataset

file_path = '/kaggle/input/companies/google_dataset.csv'

# count total lines in the file
with open(file_path) as f:
    total_lines = sum(1 for line in f)

# read the CSV, skipping bad lines
gd = pd.read_csv(file_path, low_memory=False, on_bad_lines='skip')

# copy for later ;) ... to check duplicates..
gd_copy = gd.copy()

# calculate number of lines skipped
lines_skipped = total_lines - len(gd) - 1  # subtract 1 for the header
percentage_skipped_lines = (lines_skipped/total_lines)*100

print(f"Number of lines skipped in google dataset: {lines_skipped}")
print(f"Number of total entries in google dataset: {total_lines}")
print(f"Percentage of skipped lines: {percentage_skipped_lines:.2f} %")

Number of lines skipped in google dataset: 9595
Number of total entries in google dataset: 356521
Percentage of skipped lines: 2.69 %


In [2]:
# count the number of lines skipped in facebook dataset

file_path = '/kaggle/input/companies/facebook_dataset.csv'

# count total lines in the file
with open(file_path) as f:
    total_lines = sum(1 for line in f)

# read the CSV, skipping bad lines
fd = pd.read_csv(file_path, low_memory=False, on_bad_lines='skip')

# calculate number of lines skipped
lines_skipped = total_lines - len(fd) - 1  # subtract 1 for the header
percentage_skipped_lines = (lines_skipped/total_lines)*100

print(f"Number of lines skipped in facebook dataset: {lines_skipped}")
print(f"Number of total entries in facebook dataset: {total_lines}")
print(f"Percentage of skipped lines: {percentage_skipped_lines:.2f} %")

Number of lines skipped in facebook dataset: 913
Number of total entries in facebook dataset: 72081
Percentage of skipped lines: 1.27 %


We can see that skipping the lines that had parsing errors results in eliminating only 2.69% in google dataset and 1.27% in facebook dataset of the total entries. Looks good for now but we should try to regain those lines later...

Another thing to mention is that the website dataset uses ";" to delimitate columns, but besides that, it is a really clean dataset!

In [3]:
wd = pd.read_csv('/kaggle/input/companies/website_dataset.csv', sep=';')
wd.head()

,root_domain,domain_suffix,language,legal_name,main_city,main_country,main_region,phone,site_name,tld,s_category
0,converterman.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,clothesencounter.ca,ca,en,NaN,cardigan,canada,prince edward island,13066937766,Clothes Encounter,ca,Shoes & Other Footwear Stores
2,investa.com.au,com.au,en,Investa Wholesale Funds Management Limited,brisbane,australia,queensland,61282269300,Investa Property Group,au,Real Estate Developers
3,timminsgarage.com,com,en,Timmins Garage Inc.,timmins,canada,ontario,18775896640,Timmins Garage,com,Automobile Dealers & Manufacturers
4,destinate.ca,ca,en,Destinate Group Ltd.,NaN,Canada,NaN,NaN,Destinate Group,ca,Business Consulting


# Step 2: Data Preprocessing

In order to compare values between different datasets, I need to clean all important values and for that I will proceed with the following steps:
- change data type to string
- change column names (same names for all 3 datasets)
- convert to lowercase
- remove punctuation, accents and extra spaces
- remove company suffix (inc, llc, co, srl etc.)
- handle special characters

First I will need to get a glimpse of the datasets in order to change the column names. In this way the data will be uniform and my confusion will be minimised :)

In [4]:
# check column names
gd.info()
fd.info()
wd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346925 entries, 0 to 346924
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   address             321315 non-null  object
 1   category            295862 non-null  object
 2   city                302199 non-null  object
 3   country_code        295361 non-null  object
 4   country_name        302235 non-null  object
 5   name                346893 non-null  object
 6   phone               314789 non-null  object
 7   phone_country_code  234104 non-null  object
 8   raw_address         302909 non-null  object
 9   raw_phone           318977 non-null  object
 10  region_code         302049 non-null  object
 11  region_name         302054 non-null  object
 12  text                343411 non-null  object
 13  zip_code            265646 non-null  object
 14  domain              346923 non-null  object
dtypes: object(15)
memory usage: 39.7+ MB
<class 'pandas

In [5]:
# rename columns (shared ones)
gd.rename(columns={"country_name": "country", "region_name": "region", "text": "details"}, inplace=True)
fd.rename(columns={"categories": "category", "country_name": "country", "region_name": "region"}, inplace=True)
wd.rename(columns={"root_domain": "domain", "main_city": "city", "main_country": "country", "main_region": "region", "site_name": "name", "s_category": "category"}, inplace=True)

# convert objects to strings
gd = gd.astype('string')
fd = fd.astype('string')
wd = wd.astype('string')

# convert to lowercase
gd = gd.map(lambda s: s.lower() if type(s) == str else s)
fd = fd.map(lambda s: s.lower() if type(s) == str else s)
wd = wd.map(lambda s: s.lower() if type(s) == str else s)

fd.head()

,domain,address,category,city,country_code,country,description,email,link,name,page_type,phone,phone_country_code,region_code,region,zip_code
0,euro-hygiene-34.fr,"134 rue entrepreneurs, za du vigné, 30420, cal...",<NA>,calvisson,fr,france,<NA>,<NA>,https://euro-hygiene-34.fr,euro hygiène,localbusiness,<NA>,<NA>,occ,occitanie,30420
1,lakesidehomeservices.ca,<NA>,appliance repair & maintenance|home builders &...,<NA>,ca,<NA>,<NA>,<NA>,https://lakesidehomeservices.ca,lakeside home services,localbusiness,16138318840.0,ca,<NA>,<NA>,<NA>
2,rossiterboats.com,<NA>,boats & yachts dealers|boat tours & cruises,<NA>,<NA>,<NA>,at rossiter boats we build semi-custom power b...,info@rossiterboats.com,http://rossiterboats.com,rossiter boats,localbusiness,<NA>,<NA>,<NA>,<NA>,<NA>
3,caa-architecture.com,"601 cordova st w # 270, v6b 1g1, vancouver, bc...",architects & architectural services|other engi...,vancouver,ca,canada,<NA>,office@caa-arcitecture.com,http://caa-architecture.com,chandler associates architecture inc.,localbusiness,16046873390.0,ca,bc,british columbia,v6b 1g1
4,apexsurety.ca,"unit 3 - 4 donald street, r3l 2t7, winnipeg, m...",<NA>,winnipeg,ca,canada,<NA>,<NA>,https://apexsurety.ca,apex surety & insurance ltd.,localbusiness,12045602508.0,ca,mb,manitoba,r3l 2t7


At this moment, the phone numbers in the facebook dataset were converted from object type (1.613832e+10) to string type (16138318840.0) so I need to get rid of the decimal before removing punctuation, otherwise all phone numbers will be 1 digit longer.

In [6]:
# remove decimal from phone number
fd['phone'] = fd['phone'].apply(lambda x: str(int(float(x))) if pd.notnull(x) else x)

fd.head()

,domain,address,category,city,country_code,country,description,email,link,name,page_type,phone,phone_country_code,region_code,region,zip_code
0,euro-hygiene-34.fr,"134 rue entrepreneurs, za du vigné, 30420, cal...",<NA>,calvisson,fr,france,<NA>,<NA>,https://euro-hygiene-34.fr,euro hygiène,localbusiness,<NA>,<NA>,occ,occitanie,30420
1,lakesidehomeservices.ca,<NA>,appliance repair & maintenance|home builders &...,<NA>,ca,<NA>,<NA>,<NA>,https://lakesidehomeservices.ca,lakeside home services,localbusiness,16138318840,ca,<NA>,<NA>,<NA>
2,rossiterboats.com,<NA>,boats & yachts dealers|boat tours & cruises,<NA>,<NA>,<NA>,at rossiter boats we build semi-custom power b...,info@rossiterboats.com,http://rossiterboats.com,rossiter boats,localbusiness,<NA>,<NA>,<NA>,<NA>,<NA>
3,caa-architecture.com,"601 cordova st w # 270, v6b 1g1, vancouver, bc...",architects & architectural services|other engi...,vancouver,ca,canada,<NA>,office@caa-arcitecture.com,http://caa-architecture.com,chandler associates architecture inc.,localbusiness,16046873390,ca,bc,british columbia,v6b 1g1
4,apexsurety.ca,"unit 3 - 4 donald street, r3l 2t7, winnipeg, m...",<NA>,winnipeg,ca,canada,<NA>,<NA>,https://apexsurety.ca,apex surety & insurance ltd.,localbusiness,12045602508,ca,mb,manitoba,r3l 2t7


At this moment I would like to remove punctuation but there is a catch! Usually, the pipe symbol ("|"), slash symbol ("/") and dash ("-") separate leading/ending letters so they do not have spaces before and after. Removing them would result in concatenating words which in case of 'category' column would not be ideal (in case of 'details' column it is not really relevant). The solution would be to replace any punctuation or special character with a white space. At the end I will strip all extra spaces.

I will remove punctuation only where I find it necessary: address, category, phone, city and name of the company (only on common columns that will be later used at merging the datasets).

In [7]:
# remove punctuation

import string
import re

# choose columns to remove punctuation from
cols_to_clean_gd = ['address', 'category', 'name', 'phone', 'city']
cols_to_clean_fd = ['address', 'category', 'name', 'phone', 'city']
cols_to_clean_wd = ['legal_name', 'phone', 'name', 'category', 'city']

# translation table that maps every punctuation character to None
punctuation_pattern = '[{}]'.format(re.escape(string.punctuation))

# replace punctuation with white space
for col in cols_to_clean_gd:
    gd[col] = gd[col].str.replace(punctuation_pattern, ' ', regex=True)
    
for col in cols_to_clean_fd:
    fd[col] = fd[col].str.replace(punctuation_pattern, ' ', regex=True)

for col in cols_to_clean_wd:
    wd[col] = wd[col].str.replace(punctuation_pattern, ' ', regex=True)

fd.head()

,domain,address,category,city,country_code,country,description,email,link,name,page_type,phone,phone_country_code,region_code,region,zip_code
0,euro-hygiene-34.fr,134 rue entrepreneurs za du vigné 30420 cal...,<NA>,calvisson,fr,france,<NA>,<NA>,https://euro-hygiene-34.fr,euro hygiène,localbusiness,<NA>,<NA>,occ,occitanie,30420
1,lakesidehomeservices.ca,<NA>,appliance repair maintenance home builders ...,<NA>,ca,<NA>,<NA>,<NA>,https://lakesidehomeservices.ca,lakeside home services,localbusiness,16138318840,ca,<NA>,<NA>,<NA>
2,rossiterboats.com,<NA>,boats yachts dealers boat tours cruises,<NA>,<NA>,<NA>,at rossiter boats we build semi-custom power b...,info@rossiterboats.com,http://rossiterboats.com,rossiter boats,localbusiness,<NA>,<NA>,<NA>,<NA>,<NA>
3,caa-architecture.com,601 cordova st w 270 v6b 1g1 vancouver bc...,architects architectural services other engi...,vancouver,ca,canada,<NA>,office@caa-arcitecture.com,http://caa-architecture.com,chandler associates architecture inc,localbusiness,16046873390,ca,bc,british columbia,v6b 1g1
4,apexsurety.ca,unit 3 4 donald street r3l 2t7 winnipeg m...,<NA>,winnipeg,ca,canada,<NA>,<NA>,https://apexsurety.ca,apex surety insurance ltd,localbusiness,12045602508,ca,mb,manitoba,r3l 2t7


In [8]:
# install cleanco library to remove company suffix
!pip install cleanco

In [9]:
# remove company suffix (inc/ltd/co/srl etc)
from cleanco import basename

gd['name'] = gd['name'].apply(lambda x: basename(x) if isinstance(x, str) else x)
fd['name'] = fd['name'].apply(lambda x: basename(x) if isinstance(x, str) else x)
wd['name'] = wd['name'].apply(lambda x: basename(x) if isinstance(x, str) else x)
wd['legal_name'] = wd['legal_name'].apply(lambda x: basename(x) if isinstance(x, str) else x)

# remove all spaces in phone numbers
gd['phone'] = gd['phone'].str.replace(' ', '', regex=True)
fd['phone'] = fd['phone'].str.replace(' ', '', regex=True)
wd['phone'] = wd['phone'].str.replace(' ', '', regex=True)

fd.head()

,domain,address,category,city,country_code,country,description,email,link,name,page_type,phone,phone_country_code,region_code,region,zip_code
0,euro-hygiene-34.fr,134 rue entrepreneurs za du vigné 30420 cal...,<NA>,calvisson,fr,france,<NA>,<NA>,https://euro-hygiene-34.fr,euro hygiène,localbusiness,<NA>,<NA>,occ,occitanie,30420
1,lakesidehomeservices.ca,<NA>,appliance repair maintenance home builders ...,<NA>,ca,<NA>,<NA>,<NA>,https://lakesidehomeservices.ca,lakeside home services,localbusiness,16138318840,ca,<NA>,<NA>,<NA>
2,rossiterboats.com,<NA>,boats yachts dealers boat tours cruises,<NA>,<NA>,<NA>,at rossiter boats we build semi-custom power b...,info@rossiterboats.com,http://rossiterboats.com,rossiter boats,localbusiness,<NA>,<NA>,<NA>,<NA>,<NA>
3,caa-architecture.com,601 cordova st w 270 v6b 1g1 vancouver bc...,architects architectural services other engi...,vancouver,ca,canada,<NA>,office@caa-arcitecture.com,http://caa-architecture.com,chandler associates architecture,localbusiness,16046873390,ca,bc,british columbia,v6b 1g1
4,apexsurety.ca,unit 3 4 donald street r3l 2t7 winnipeg m...,<NA>,winnipeg,ca,canada,<NA>,<NA>,https://apexsurety.ca,apex surety insurance,localbusiness,12045602508,ca,mb,manitoba,r3l 2t7


In [10]:
# remove leading and trailing spaces
gd = gd.map(lambda x: x.strip() if isinstance(x, str) else x)
fd = fd.map(lambda x: x.strip() if isinstance(x, str) else x)
wd = wd.map(lambda x: x.strip() if isinstance(x, str) else x)

# remove extra spaces within string
gd = gd.map(lambda x: ' '.join(x.split()) if isinstance(x, str) else x)
fd = fd.map(lambda x: ' '.join(x.split()) if isinstance(x, str) else x)
wd = wd.map(lambda x: ' '.join(x.split()) if isinstance(x, str) else x)

The last thing I would like to do is to remove accents and handle special characters. While going through the datasets, I saw that it includes non-ASCII characters (like chinese alphabet) as well, so I would like to keep those intact.

In [11]:
# remove accents but keep non-ASCII characters from other languages

import unicodedata

def strip_accents(text):
    if pd.isna(text):
        # The value is NaN, return it as is
        return text
    else:
        # Normalize the text to decompose accented characters
        text = unicodedata.normalize('NFD', text)
        # Keep only characters that do not have a combining mark
        text = ''.join(c for c in text if unicodedata.combining(c) == 0)
        return str(text)

# apply the function to certain columns
cols_to_clean = ['address', 'category', 'name']

for col in cols_to_clean:
    gd[col] = gd[col].apply(strip_accents)
    fd[col] = fd[col].apply(strip_accents)
    
for col in cols_to_clean[1:3]:
    wd[col] = wd[col].apply(strip_accents)
    
fd.head()

,domain,address,category,city,country_code,country,description,email,link,name,page_type,phone,phone_country_code,region_code,region,zip_code
0,euro-hygiene-34.fr,134 rue entrepreneurs za du vigne 30420 calvis...,<NA>,calvisson,fr,france,<NA>,<NA>,https://euro-hygiene-34.fr,euro hygiene,localbusiness,<NA>,<NA>,occ,occitanie,30420
1,lakesidehomeservices.ca,<NA>,appliance repair maintenance home builders ren...,<NA>,ca,<NA>,<NA>,<NA>,https://lakesidehomeservices.ca,lakeside home services,localbusiness,16138318840,ca,<NA>,<NA>,<NA>
2,rossiterboats.com,<NA>,boats yachts dealers boat tours cruises,<NA>,<NA>,<NA>,at rossiter boats we build semi-custom power b...,info@rossiterboats.com,http://rossiterboats.com,rossiter boats,localbusiness,<NA>,<NA>,<NA>,<NA>,<NA>
3,caa-architecture.com,601 cordova st w 270 v6b 1g1 vancouver bc cana...,architects architectural services other engine...,vancouver,ca,canada,<NA>,office@caa-arcitecture.com,http://caa-architecture.com,chandler associates architecture,localbusiness,16046873390,ca,bc,british columbia,v6b 1g1
4,apexsurety.ca,unit 3 4 donald street r3l 2t7 winnipeg mb can...,<NA>,winnipeg,ca,canada,<NA>,<NA>,https://apexsurety.ca,apex surety insurance,localbusiness,12045602508,ca,mb,manitoba,r3l 2t7


Now I would like to check if non-ASCII characters where left unchanged by using an example from the facebook dataset.

In [12]:
value = '寶血女子中學'
print(fd.loc[fd['name'] == value])

          domain                                       address  \
233  pbss.edu.hk  柴灣新廈街338號 0000 hong kong hong kong hong kong   

               category  city country_code    country description email  \
233  elementary schools  <NA>           hk  hong kong        <NA>  <NA>   

                    link    name     page_type phone phone_country_code  \
233  https://pbss.edu.hk  寶血女子中學  organization  <NA>               <NA>   

    region_code region zip_code  
233        <NA>   <NA>     <NA>  


# Step 3: Exploratory Data Analysis

Now that all important data is uniform I can explore the datasets as follows:
- check duplicates and null values
- decide to keep or remove duplicates (based on the column)
- decide which are the best columns to use when joining

The best columns to use when joining should be the ones that have the most unique values and a low count of null values (preferably none).

In [13]:
print(f"The shape of the google dataset is: {gd.shape} (total entries, total columns)")
print(f"The shape of the facebook dataset is: {fd.shape}")
print(f"The shape of the website dataset is: {wd.shape}")

The shape of the google dataset is: (346925, 15) (total entries, total columns)
The shape of the facebook dataset is: (71167, 16)
The shape of the website dataset is: (72018, 11)


I found that the Google dataset had by far the most total entries. The Facebook and Website datasets had a very similar number of total entries.

Next, let's check null values and unique values. I will put all 3 datasets one next to the other for an easier comparison.

In [14]:
# calculate number of unique values for each dataset
isnull_gd = gd.isnull().sum()
isnull_fd = fd.isnull().sum()
isnull_wd = wd.isnull().sum()

# concatenate the results into a single DataFrame
result1 = pd.concat([isnull_gd, isnull_fd, isnull_wd], axis=1)

# change display option
pd.options.display.float_format = '{:.0f}'.format

# rename the columns
result1.columns = ['Google', 'Facebook', 'Website']

print(f"The sum of null values for each column:\n\n{result1}")

The sum of null values for each column:

                    Google  Facebook  Website
address              25610     14477      NaN
category             51063     16574     1384
city                 44726     26600    11052
country_code         51564     13976      NaN
country              44690     26302     7149
name                    32         0     3304
phone                32136     26818     6378
phone_country_code  112821     33616      NaN
raw_address          44016       NaN      NaN
raw_phone            27948       NaN      NaN
region_code          44876     26605      NaN
region               44871     26605    11085
details               3514       NaN      NaN
zip_code             81279     35544      NaN
domain                   2         0        1
description            NaN     43978      NaN
email                  NaN     51414      NaN
link                   NaN         0      NaN
page_type              NaN         4      NaN
domain_suffix          NaN       NaN   

In [15]:
# calculate number of unique values for each dataset
nunique_gd = gd.nunique()
nunique_fd = fd.nunique()
nunique_wd = wd.nunique()

# concatenate the results into a single DataFrame
result2 = pd.concat([nunique_gd, nunique_fd, nunique_wd], axis=1)

# change display option
pd.options.display.float_format = '{:.0f}'.format

# rename the columns
result2.columns = ['Google', 'Facebook', 'Website']

print(f"The sum of unique values for each column:\n\n{result2}")

The sum of unique values for each column:

                    Google  Facebook  Website
address             274317     56357      NaN
category               468     10073      562
city                 22048      6325     9594
country_code           213       130      NaN
country                187       116      223
name                338305     70950    67394
phone               277563     44272    60053
phone_country_code     212       100      NaN
raw_address         137626       NaN      NaN
raw_phone           284780       NaN      NaN
region_code            678       434      NaN
region                1502       681     1114
details             339455       NaN      NaN
zip_code            100697     25958      NaN
domain               70109     71167    72017
description            NaN     26807      NaN
email                  NaN     19745      NaN
link                   NaN     71167      NaN
page_type              NaN         4      NaN
domain_suffix          NaN       NaN 

At the moment, we can make the following statements:
1. 'domain' and 'name' columns are the ones that have the least amount of null values in all 3 datasets, which is an important aspect when joining datasets. 'domain' is the winner for now.
2. Values in 'domain' column in website dataset are all unique (except 1 null value).
3. Values in 'domain' column in facebook dataset are all unique.
4. Google dataset has a lot of duplicates in 'domain' column so for this particular dataset maybe we need to find another identification key. The 'name' column has the biggest amount of unique values.

In website dataset there are 2 columns that represent the name of the company: the legal name and the site name (most used/known). To decrease the number of null values in 'name' column (3304 at the moment), I decided to copy the values from 'legal_name' to 'name' if 'name' value is null. I do this because I intend to use 'name' column as merge column later on.

In [16]:
initial_null_names_count = wd['name'].isnull().sum()

# copy 'legal_name' values to 'name' values if 'name' value is null
wd['name'] = wd['name'].combine_first(wd['legal_name'])

print(f"Let's see if the number of null values in 'name' column decreased: {wd['name'].isnull().sum()} versus {initial_null_names_count} (initial).")

Let's see if the number of null values in 'name' column decreased: 2619 versus 3304 (initial).


A bit better I would say!

Next step: check duplicates and decide whether they should be kept or removed. I will start by checking the 'domain' column first for all 3 datasets. As we know so far, website and facebook datasets do not have duplicates in this column so I will check only the google dataset.

In [17]:
# check duplicates in 'domain' column in google dataset
duplicate_counts_domain_gd = gd['domain'].value_counts()
duplicate_counts_domain_gd = duplicate_counts_domain_gd[duplicate_counts_domain_gd > 1]

print(duplicate_counts_domain_gd)

domain
facebook.com                 69465
postoffice.co.uk              5872
instagram.com                 5463
ihg.com                       4192
marriott.com                  3816
                             ...  
cliniqueauditivebougie.ca        2
polarvision.ca                   2
montserrat.edu                   2
nextflightcourier.com            2
artisansofmedicine.com           2
Name: count, Length: 14987, dtype: int64


Now we can clearly see that the google dataset has a ton of duplicates in the 'domain' column. Even though there are many duplicates, the entries seem to represent different companies. For example, many companies may have 'facebook.com' as their domain.

Let's check how many duplicates does the 'name' column have.

In [18]:
# check duplicates in 'name' column in google dataset
duplicate_counts_name_gd = gd['name'].value_counts()
duplicate_counts_name_gd = duplicate_counts_name_gd[duplicate_counts_name_gd > 1]

print(duplicate_counts_name_gd)

name
lincoln elementary school                      21
transportation department                      16
city hall                                      15
washington elementary school                   15
atm                                            14
                                               ..
meridian development                            2
heart to home meals                             2
st paul s lutheran church school                2
home instead familien und seniorenbetreuung     2
central home realty                             2
Name: count, Length: 6260, dtype: int64


Strange...let's see if they represent the same companies!

In [19]:
# check the address for every company that has the same name
address = gd.loc[gd['name'] == 'lincoln elementary school', 'address']
print(address)

21504                    705 california st salinas ca 93901
46528                           701 5th ave dayton ky 41074
74484     712 lincoln ave ridgefield park nj 07660 unite...
75549     746 n maple grove ave hudson mi 49247 united s...
82063                         1812 cron st augusta ks 67010
117536                         1725 13th st gering ne 69341
147761                 15615 clifton blvd lakewood oh 44107
170635                          930 s sirrine mesa az 85210
173041                66 bartlett st new brunswick nj 08901
200356                    300 teeters ct iowa city ia 52246
205755                  501 s euclid ave princeton il 61356
206942                 720 s franklin ave hastings ne 68901
212697                                   sault ste marie mi
221379                    600 5th ave charles city ia 50616
261957                 3312 n douglas ave loveland co 80538
262716                    60 e lincoln ave zeeland mi 49464
263050                         2120 9th 

Same name for 21 different companies. Now we know that we cannot remove 'name' duplicates that easily. In this case, when joining, there will be some extra conditions (ex: name + city).

This might be a good time to drop any full duplicates.

In [20]:
# count and drop duplicates
original_length_gd = len(gd)
original_length_fd = len(fd)
original_length_wd = len(wd)

# print duplicates before dropping
print("Duplicates in google dataset:")
print(gd[gd.duplicated()])
print("\n\nDuplicates in facebook dataset:")
print(fd[fd.duplicated()])
print("\n\nDuplicates in website dataset:")
print(wd[wd.duplicated()])

gd = gd.drop_duplicates()
fd = fd.drop_duplicates()
wd = wd.drop_duplicates()

new_length_gd = len(gd)
new_length_fd = len(fd)
new_length_wd = len(wd)

num_duplicates_gd = original_length_gd - new_length_gd
num_duplicates_fd = original_length_fd - new_length_fd
num_duplicates_wd = original_length_wd - new_length_wd

print(f"\n\nNumber of duplicates dropped in google dataset: {num_duplicates_gd}")
print(f"Number of duplicates dropped in facebook dataset: {num_duplicates_fd}")
print(f"Number of duplicates dropped in website dataset: {num_duplicates_wd}")

Duplicates in google dataset:
                                                  address  \
84383   133 high st bitton bristol bs30 6hq united kin...   
137349  551 rue saint georges saint jerome qc j7z 5b7 ...   
165571         10101 111 st nw edmonton ab t5k 1k6 canada   
195938          20420 hwy 11 s bradford on l3z 2b7 canada   
226386                                               <NA>   
244335  2727 n holland sylvania rd k toledo oh 43615 u...   
271954         10107 111 st nw edmonton ab t5k 2z1 canada   
301204                                               <NA>   
327540                                               <NA>   
328942                                               <NA>   
336727         10103 111 st nw edmonton ab t5k 2y1 canada   

                                         category          city country_code  \
84383                                        <NA>       bristol           gb   
137349                     malls shopping centers  saint jerome           ca 

Perfect! The last thing I would like to do to the google dataset is to check if the duplicates are valid duplicates or maybe I did something wrong on the way :O I will check them by 'raw_phone' column because it has the least amount of null values and it is one of the few columns I left unchanged by now :)

I took all raw phone numbers from the duplicates found and checked them in the original google dataset (a copy of it).

In [21]:
# choose values from 'raw_phone' column to check duplicates
phones = ['+44 117 932 1990', '+1 450-304-2669', '+1 780-709-0728', '+1 905-775-7876', '+1 419-725-9223', '+33 7 88 28 60 84']

# print lines that contain the phone values
print("Lines in google dataset that contain the raw_phone values (separated by dashes):")
for phone in phones:
    print(gd_copy[gd_copy['raw_phone'] == phone])
    print('-' * 100)

Lines in google dataset that contain the raw_phone values (separated by dashes):
                                                 address category     city  \
67278  133 High St, Bitton, Bristol BS30 6HQ, United ...      NaN  bristol   
84383  133 High St, Bitton, Bristol BS30 6HQ, United ...      NaN  bristol   

      country_code    country_name                                name  \
67278           gb  united kingdom  Premier - Bitton Convenience Store   
84383           gb  united kingdom   Premier- Bitton Convenience Store   

               phone phone_country_code  \
67278  +441179321990                 gb   
84383  +441179321990                 gb   

                                      raw_address         raw_phone  \
67278  Bristol, United Kingdom · +44 117 932 1990  +44 117 932 1990   
84383  Bristol, United Kingdom · +44 117 932 1990  +44 117 932 1990   

      region_code region_name  \
67278         eng     england   
84383         eng     england   

                 

Huh! Great news! By removing special characters, accents, extra spaces and by converting everything to lowercase, we found some duplicates (11). It means that the data is on good hands :)

Next I am going through the same process with the facebook and website dataset: check duplicates and decide on keeping/removing them. Keep in mind that 'domain' values are unique so probably I am not going to remove anything.

In [22]:
# check duplicates in 'name' column in facebook dataset
duplicate_counts_name_fd = fd['name'].value_counts()
duplicate_counts_name_fd = duplicate_counts_name_fd[duplicate_counts_name_fd > 1]

print(duplicate_counts_name_fd)

name
nissan                                 7
hilti                                  6
little learners preschool              4
grace christian school                 4
jll                                    4
                                      ..
sacred heart catholic school           2
city of neodesha                       2
absolute contracting                   2
our lady of victory catholic school    2
cornerstone assembly of god            2
Name: count, Length: 173, dtype: int64


In [23]:
# check the domain for every company that has the same name
nissan_domains = fd.loc[fd['name'] == 'nissan', 'domain']
print(nissan_domains)

27521    nissan.com.tr
28728    nissan.com.hk
38564    nissan.com.mx
58699        nissan.ru
61938    nissan-me.com
65464     nissan.co.za
67170    nissan.com.au
Name: domain, dtype: object


Despite having the same name, the companies are different because of the country in which they are located. No dropping here either!

In [24]:
# check duplicates in 'name' column in website dataset
duplicate_counts_name_wd = wd['name'].value_counts()
duplicate_counts_name_wd = duplicate_counts_name_wd[duplicate_counts_name_wd > 1]

print(duplicate_counts_name_wd)

name
mysite                      69
                            33
the canadian real estate    27
home                        26
inmotion hosting            16
                            ..
mirvac                       2
our lady of sorrows          2
at a glance                  2
april marine                 2
cornerstone insurance        2
Name: count, Length: 850, dtype: int64


In [25]:
# check the domain for every company that has the same name
mysite_domains = wd.loc[wd['name'] == 'mysite', 'domain']
print(mysite_domains)

881               blackstackmechanical.ca
1501           performanceautocalgary.com
3077                 marinelifecanada.com
4827                      freshlookbl.com
5691             ontariolegalservices.com
                       ...               
69307    thechristiantabernaclechurch.com
69420                    dreamdenture.com
70949                        luvmydog.net
71059              connorstreeservice.com
71427                themetalmonkey.co.uk
Name: domain, Length: 69, dtype: object


Last but not least, I will introduce index columns and suffixes for all columns in all 3 datasets. This is a prestep to make the join easier.

In [26]:
# add index column for each dataset
gd = gd.reset_index()
fd = fd.reset_index()
wd = wd.reset_index()

# add suffixes for all columns
gd = gd.add_suffix('_g')
fd = fd.add_suffix('_f')
wd = wd.add_suffix('_w')

# set index column for each dataset
gd.set_index('index_g', inplace=True)
fd.set_index('index_f', inplace=True)
wd.set_index('index_w', inplace=True)

# Step 4: Datasets join

Now I am going to join the datasets. My plan is to first join the facebook and website datasets by the 'domain' column as it is the column that has the least amount of null values and the biggest amount of unique values for both datasets.

Best method I encountered so far is using the Python Record Linkage Toolkit. This library has the capability to limit the pool of potential matches by using the so called blockers. In this way, by blocking the domain for each company, the number of matches will be significantly reduced. Having in mind only the common columns (country, city, region, name, phone, domain, category) I will choose only 4 of them to compare using the methods defined in the record linkage library. I will use an exact comparison for the columns that have exact values such as country, city and region. For the name column I will use a string similarity comparison, as the names can differ in many ways (abbreviations, different language, stop words etc). The method for the string similarity comparison that I will be using is jarowinkler. The threshold needs to be evaluated, but it surely needs to be on the higher edge. One important note: the way null values are proccessed needs to be adressed. In my case, the comparison between a null value and a non-null value must be true (score = 1) for any column or comparison method used. So only if the values that are compared are the same or one of them is null, that is the only time the score will be 1 for that position (row, column). In this way, the score for a match will be in a range from 0 to 4 (because it compares 4 columns) and I will take into consideration only the best scores (> 3).

In [27]:
!pip install recordlinkage

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.9/926.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.6 MB/s eta 0:00:00


In [28]:
import recordlinkage

# create indexer object to identify pairs of rows
indexer = recordlinkage.Index()

# apply blocking on 'domain_f' and 'domain_w'
indexer.block(left_on='domain_f', right_on='domain_w')

# generate candidate links
candidates = indexer.index(fd, wd)

print(f"Number of potential matches: {len(candidates)}")

Number of potential matches: 71162


As we can see, by blocking the domain column, the number of matches is very small compared with the total number of potential matches (nr. entries facebook * nr. entries website). We saved a lot of time and computational resources!

Next I will compute the comparisons (4 columns -> 4 comparisons) and I will track the runtime to see how long it takes.

In [29]:
# create compare object
compare = recordlinkage.Compare()

# compare country exactly
compare.exact('country_f', 'country_w', label='country_score', missing_value=1)

# compare city exactly
compare.exact('city_f', 'city_w', label='city_score', missing_value=1)

# compare region exactly
compare.exact('region_f', 'region_w', label='region_score', missing_value=1)

# compare name using Jaro-Winkler similarity
compare.string('name_f',
            'name_w',
            method='jarowinkler',
            threshold=0.85,
            label='name_score',
            missing_value=1)

import time

# record start time
t = time.time()

# compute feature vectors for candidate links
features = compare.compute(candidates, fd, wd)

# calculate elapsed time
interval = time.time() - t

print(f"Computing time: {interval} seconds")

Computing time: 0.5570087432861328 seconds


The features dataframe should contain the index of the rows that matched (both facebook index and website index) and the score for each comparison. They are labeled country_score, city_score, region_score, name_score and they can take only 1 and 0 as values (true or false).

In [30]:
features.head()

,,country_score,city_score,region_score,name_score
index_f,index_w,,,,
0,64518,1,0,1,1
1,70546,1,1,1,1
2,14350,1,1,1,1
3,22282,1,1,1,1
4,66294,1,1,1,0


In [31]:
print(f"Score | Nr.of matches")

# sum features by score, count unique sums and sort in descending order
features.sum(axis=1).value_counts().sort_index(ascending=False)

Score | Nr.of matches


4    48140
3    17245
2     2889
1     2282
0      606
Name: count, dtype: int64

Now I have to decide with what score I will go further. Country, region and city should require exact matches such that companies located in different regions are treated as different companies. Name is the feature that is compared using similarity and in this case, a score of 0 would mean that the values are more different than similar, so probably not the same company either. Having this in mind, the score that I find most accurate will be score=4 which means 48.140 matches (from a total of 143.185 entries) from facebook and website dataset. This means that the match rate is around 67%. Looks good!

Now we merge!!!

In [32]:
# filter rows where the sum along the row is greater than or equal to 4
filtered = features[features.sum(axis=1) >= 4] 

filtered.reset_index(inplace=True)

# merge 'filtered' and 'fd' on 'index_f', keeping all rows from both
merged = pd.merge(filtered, fd, left_on='index_f', right_index=True, how='outer')

# merge the resulting DataFrame with 'wd' on 'index_w', keeping all rows from both
merged = pd.merge(merged, wd, left_on='index_w', right_index=True, how='outer')

# merge common columns (_f and _w)
merged['city'] = merged['city_f'].combine_first(merged['city_w'])
merged['country'] = merged['country_f'].combine_first(merged['country_w'])
merged['region'] = merged['region_f'].combine_first(merged['region_w'])
merged['domain'] = merged['domain_f'].combine_first(merged['domain_w'])

# keep website 'phone' column (or facebook if website is null)
merged['phone'] = merged['phone_w'].fillna(merged['phone_f'])

# keep website 'name' column (or facebook if website is null)
merged['name'] = merged['name_w'].fillna(merged['name_f'])

# keep website 'category' column (or facebook if website is null)
merged['category'] = merged['category_w'].fillna(merged['category_f'])

# drop columns (_f and _w)
merged = merged.drop(['index_f', 'index_w', 'country_score', 'city_score', 'region_score', 'name_score',
                      'city_f', 'country_f', 'name_f', 'phone_f', 'region_f', 'category_f', 'domain_f',
                      'city_w', 'country_w', 'name_w', 'phone_w', 'region_w', 'category_w', 'domain_w' ], axis=1)

merged.head()

,address_f,country_code_f,description_f,email_f,link_f,page_type_f,phone_country_code_f,region_code_f,zip_code_f,domain_suffix_w,language_w,legal_name_w,tld_w,city,country,region,domain,phone,name,category
42751,246 brockport dr m9w 5s1 toronto on canada ont...,ca,converter man limited is a leader in torque co...,peter@converterman.com,http://converterman.com,localbusiness,ca,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,converterman.com,14166747000,converter man,auto restoration service
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ca,en,<NA>,ca,cardigan,canada,prince edward island,clothesencounter.ca,13066937766,clothes encounter,shoes other footwear stores
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,com.au,en,investa wholesale funds management,au,brisbane,australia,queensland,investa.com.au,61282269300,investa property group,real estate developers
44608,1395 riverside drive p4r 1a6 timmins on canada...,ca,"timmins garage incorporated is a chevrolet, gm...",contactus@timminsgarage.com,http://timminsgarage.com,localbusiness,<NA>,on,p4r 1a6,com,en,timmins garage,com,timmins,canada,ontario,timminsgarage.com,18775896640,timmins garage,automobile dealers manufacturers
34731,2256 vista oak rd oakville on canada ontario,ca,"destinate provides real-world, results-focused...",info@destinate.ca,http://destinate.ca,organization,<NA>,on,<NA>,ca,en,destinate group,ca,oakville,canada,ontario,destinate.ca,18662929696,destinate group,business consulting


In [33]:
# check if there are any lines that have null values in the domain column
null_rows = merged[merged['domain'].isnull()]
print(f"Number of entries with null domain: {len(null_rows)}\n")
print(null_rows)

Number of entries with null domain: 1

    address_f country_code_f description_f email_f link_f page_type_f  \
NaN       NaN            NaN           NaN     NaN    NaN         NaN   

    phone_country_code_f region_code_f zip_code_f domain_suffix_w language_w  \
NaN                  NaN           NaN        NaN            <NA>       <NA>   

    legal_name_w tld_w  city country region domain phone name category  
NaN         <NA>  <NA>  <NA>    <NA>   <NA>   <NA>   NaN  NaN      NaN  


We found 1 entry with null domain. It seems that all columns are null, so we will remove this entry.

In [34]:
# drop the column that was found above
merged = merged.dropna(subset=['domain'])

# rename columns, dataset and add index column (prepare for the next merge with google dataset)
merged.columns = ['adress', 'country_code', 'description', 'email', 'link', 'page_type',
                  'phone_country_code', 'region_code', 'zip_code', 'domain_suffix', 'language',
                  'legal_name', 'tld', 'city', 'country', 'region', 'domain', 'phone', 'name', 'category']
merged_f_w = merged.reset_index()
merged_f_w = merged_f_w.drop(['index'], axis=1) # remove the indexation made by the notebook???
merged_f_w = merged_f_w.reset_index()

# add suffixes to be easily distinguished (_f_w)
merged_f_w = merged_f_w.add_suffix('_f_w')

# set index column for the new dataset
merged_f_w.set_index('index_f_w', inplace=True)

merged_f_w[['city_f_w', 'country_f_w', 'region_f_w', 'domain_f_w', 'phone_f_w', 'name_f_w', 'category_f_w']].head()

,city_f_w,country_f_w,region_f_w,domain_f_w,phone_f_w,name_f_w,category_f_w
index_f_w,,,,,,,
0,<NA>,<NA>,<NA>,converterman.com,14166747000,converter man,auto restoration service
1,cardigan,canada,prince edward island,clothesencounter.ca,13066937766,clothes encounter,shoes other footwear stores
2,brisbane,australia,queensland,investa.com.au,61282269300,investa property group,real estate developers
3,timmins,canada,ontario,timminsgarage.com,18775896640,timmins garage,automobile dealers manufacturers
4,oakville,canada,ontario,destinate.ca,18662929696,destinate group,business consulting


Last merge: Facebook-Website dataset and Google dataset

In [35]:
# create indexer object to identify pairs of rows
indexer = recordlinkage.Index()

# apply blocking on 'domain_f_w' and 'domain_g'
indexer.block(left_on='domain_f_w', right_on='domain_g')

# generate candidate links
candidates = indexer.index(merged_f_w, gd)

print(f"Number of potential matches: {len(candidates)}")

Number of potential matches: 507605


In [36]:
# create compare object
compare = recordlinkage.Compare()

# compare country exactly
compare.exact('country_f_w', 'country_g', label='country_score', missing_value=1)

# compare city exactly
compare.exact('city_f_w', 'city_g', label='city_score', missing_value=1)

# compare region exactly
compare.exact('region_f_w', 'region_g', label='region_score', missing_value=1)

# compare name using Jaro-Winkler similarity
compare.string('name_f_w',
            'name_g',
            method='jarowinkler',
            threshold=0.85,
            label='name_score',
            missing_value=1)

import time

# record start time
t = time.time()

# compute feature vectors for candidate links
features = compare.compute(candidates, merged_f_w, gd)

# calculate elapsed time
interval = time.time() - t

print(f"Computing time: {interval} seconds")

Computing time: 3.874389171600342 seconds


In [37]:
features.head()

country_score  city_score  region_score  name_score
index_f_w index_g                                                     
0         283561               1           1             1           1
1         122511               1           0             0           1
2         75533                1           0             0           0
          87518                1           0             0           0
          184991               1           0             0           0

In [38]:
print(f"Score | Nr.of matches")

# sum features by score, count unique sums and sort in descending order
features.sum(axis=1).value_counts().sort_index(ascending=False)

Score | Nr.of matches


4     73891
3    168720
2     90720
1    104165
0     70109
Name: count, dtype: int64

In [39]:
# filter rows where the sum along the row is greater than or equal to 4
filtered = features[features.sum(axis=1) >= 4] 

filtered.reset_index(inplace=True)

# merge 'filtered' and 'merged_f_w' on 'index_f_w', keeping all rows from both
merged = pd.merge(filtered, merged_f_w, left_on='index_f_w', right_index=True, how='outer')

# merge the resulting DataFrame with 'gd' on 'index_g', keeping all rows from both
merged = pd.merge(merged, gd, left_on='index_g', right_index=True, how='outer')

# merge common columns (_f_w and _g)
merged['city'] = merged['city_f_w'].combine_first(merged['city_g'])
merged['country'] = merged['country_f_w'].combine_first(merged['country_g'])
merged['region'] = merged['region_f_w'].combine_first(merged['region_g'])
merged['domain'] = merged['domain_f_w'].combine_first(merged['domain_g'])

# keep facebook-website 'phone' column (or google if facebook-website is null)
merged['phone'] = merged['phone_f_w'].fillna(merged['phone_g'])

# keep facebook-website 'name' column (or google if facebook-website is null)
merged['name'] = merged['name_f_w'].fillna(merged['name_g'])

# keep facebook-website 'category' column (or google if facebook-website is null)
merged['category'] = merged['category_f_w'].fillna(merged['category_g'])

# drop columns (_f_w and _g)
merged = merged.drop(['index_f_w', 'index_g', 'country_score', 'city_score', 'region_score', 'name_score',
                      'city_f_w', 'country_f_w', 'name_f_w', 'phone_f_w', 'region_f_w', 'category_f_w', 'domain_f_w',
                      'city_g', 'country_g', 'name_g', 'phone_g', 'region_g', 'category_g', 'domain_g' ], axis=1)
merged.head()

,adress_f_w,country_code_f_w,description_f_w,email_f_w,link_f_w,page_type_f_w,phone_country_code_f_w,region_code_f_w,zip_code_f_w,domain_suffix_f_w,...,region_code_g,details_g,zip_code_g,city,country,region,domain,phone,name,category
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,nsw,"4.1 (766) · craft store west gosford nsw, aust...",2250,gosford,australia,new south wales,spotlightstores.com,61243355946,spotlight west gosford,fabric based home goods
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,on,4.7 (100) · book store 400 scott st · in grant...,l2m 3w2,st catharines,canada,ontario,bookmanager.com,19059374553,heritage christian book store,book stores
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,ns,4.7 (40) · building materials store 7+ years i...,b5a 2j9,yarmouth,canada,nova scotia,timbermart.ca,19027429181,pleasant timber mart,other building material retailers
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,ca,4.3 (15) · medical spa 7+ years in business · ...,90025,los angeles,united states,california,linktr.ee,18184268353,skin specifics medical spa west la,plastic surgery clinics
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,<NA>,5.0 (1) · mobile caterer open ⋅ closes 9pm,<NA>,<NA>,<NA>,<NA>,linktr.ee,<NA>,lakay express,catering delivery


In [40]:
# merge common columns that were not compared but have exact values (_f_w and _g)
merged['zip_code'] = merged['zip_code_f_w'].combine_first(merged['zip_code_g'])
merged['country_code'] = merged['country_code_f_w'].combine_first(merged['country_code_g'])
merged['phone_country_code'] = merged['phone_country_code_f_w'].combine_first(merged['phone_country_code_g'])
merged['region_code'] = merged['region_code_f_w'].combine_first(merged['region_code_g'])

# drop columns (_f_w and _g)
merged = merged.drop(['zip_code_f_w', 'zip_code_g', 'country_code_f_w', 'country_code_g', 'phone_country_code_f_w',
                      'phone_country_code_g', 'region_code_f_w', 'region_code_g', 'adress_f_w'], axis=1)

# rename columns in final dataset
merged = merged.rename(columns={'description_f_w': 'description', 'email_f_w': 'email', 'link_f_w': 'link',
                       'page_type_f_w': 'page_type', 'domain_suffix_f_w': 'domain_suffix', 'language_f_w': 'language',
                       'legal_name_f_w': 'legal_name', 'tld_f_w': 'tld', 'address_g': 'address', 'raw_address_g': 'raw_address',
                       'raw_phone_g': 'raw_phone', 'details_g': 'details'})

# reorder columns in final dataset
merged = merged[['name', 'category', 'country', 'region', 'city', 'phone', 'domain', 'address', 'description',
                 'details', 'email', 'link', 'zip_code', 'country_code', 'region_code', 'phone_country_code','language',
                 'legal_name', 'page_type', 'domain_suffix', 'tld', 'raw_address', 'raw_phone']]

# rename final dataset
final_merge = merged

# reset index
final_merge.reset_index(inplace=True)
final_merge = final_merge.drop(['index'], axis=1)

# count and drop duplicates
original_length = len(final_merge)

final_merge = final_merge.drop_duplicates()
new_length = len(final_merge)
num_duplicates = original_length - new_length
print(f"Number of duplicates dropped in final dataset: {num_duplicates}")

Number of duplicates dropped in final dataset: 220


In [41]:
# save the final dataset
final_merge.to_csv('final_datasets_join.csv')

final_merge.head(50)

,name,category,country,region,city,phone,domain,address,description,details,...,country_code,region_code,phone_country_code,language,legal_name,page_type,domain_suffix,tld,raw_address,raw_phone
0,spotlight west gosford,fabric based home goods,australia,new south wales,gosford,61243355946,spotlightstores.com,28 central coast hwy west gosford nsw 2250 aus...,NaN,"4.1 (766) · craft store west gosford nsw, aust...",...,au,nsw,au,NaN,NaN,NaN,NaN,NaN,"west gosford nsw, australia",+61 2 4335 5946
1,heritage christian book store,book stores,canada,ontario,st catharines,19059374553,bookmanager.com,400 scott st st catharines on l2m 3w2 canada,NaN,4.7 (100) · book store 400 scott st · in grant...,...,ca,on,ca,NaN,NaN,NaN,NaN,NaN,400 scott st · in grantham plaza,+1 905-937-4553
2,pleasant timber mart,other building material retailers,canada,nova scotia,yarmouth,19027429181,timbermart.ca,191 pleasant st yarmouth ns b5a 2j9 canada,NaN,4.7 (40) · building materials store 7+ years i...,...,ca,ns,ca,NaN,NaN,NaN,NaN,NaN,"7+ years in business · yarmouth, ns, canada",+1 902-742-9181
3,skin specifics medical spa west la,plastic surgery clinics,united states,california,los angeles,18184268353,linktr.ee,11040 santa monica blvd suite 370 los angeles ...,NaN,4.3 (15) · medical spa 7+ years in business · ...,...,us,ca,us,NaN,NaN,NaN,NaN,NaN,7+ years in business · 11040 santa monica blvd...,+1 818-426-8353
4,lakay express,catering delivery,<NA>,<NA>,<NA>,<NA>,linktr.ee,<NA>,NaN,5.0 (1) · mobile caterer open ⋅ closes 9pm,...,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>
5,b nektar meadery taproom headquarters,pubs bars,united states,michigan,ferndale,13137446323,linktr.ee,ferndale mi united states,NaN,"4.8 (296) · $$ · bar ferndale, mi, united stat...",...,us,mi,us,NaN,NaN,NaN,NaN,NaN,"ferndale, mi, united states",+1 313-744-6323
6,icare,health insurance agency,australia,new south wales,sydney,61134422,icare.nsw.gov.au,321 kent st sydney nsw 2000 australia,matt’s story: living with a spinal cord injury 7,"2.3 (187) · insurance company sydney nsw, aust...",...,au,nsw,<NA>,en,<NA>,organization,gov.au,au,"sydney nsw, australia",<NA>
7,south cleveland avenue kindercare,preschools kindergartens,united states,ohio,westerville,16148990026,kindercare.com,55 s cleveland ave westerville oh 43081,NaN,3.6 (20) · preschool 7+ years in business · we...,...,us,oh,<NA>,NaN,NaN,NaN,NaN,NaN,"7+ years in business · westerville, oh",(614) 899-0026
8,torrey pines kindercare,preschools kindergartens,united states,nevada,las vegas,17023670822,kindercare.com,4050 s torrey pines dr las vegas nv 89103 unit...,NaN,3.8 (11) · preschool 7+ years in business · la...,...,us,nv,us,NaN,NaN,NaN,NaN,NaN,"7+ years in business · las vegas, nv, united s...",+1 702-367-0822
9,meadowlands kindercare,preschools kindergartens,<NA>,<NA>,<NA>,16514296316,kindercare.com,7 years in business · st paul mn united states,NaN,4.8 (9) · kindergarten 1000 meadowlands dr clo...,...,us,<NA>,us,NaN,NaN,NaN,NaN,NaN,"7+ years in business · st paul, mn, united states",(651) 429-6316


In [42]:
final_merge.info()

<class 'pandas.core.frame.DataFrame'>
Index: 387282 entries, 0 to 387501
Data columns (total 23 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   name                387243 non-null  object
 1   category            341918 non-null  object
 2   country             348854 non-null  object
 3   region              346634 non-null  object
 4   city                346811 non-null  object
 5   phone               351827 non-null  object
 6   domain              387280 non-null  object
 7   address             322392 non-null  object
 8   description         30619 non-null   object
 9   details             345330 non-null  object
 10  email               21069 non-null   object
 11  link                85243 non-null   object
 12  zip_code            279303 non-null  object
 13  country_code        314771 non-null  object
 14  region_code         317354 non-null  object
 15  phone_country_code  246033 non-null  object
 16  languag